ML Classification Lession 1


In [3]:
#import libraries
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Check distribution of target variable
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

In [6]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [7]:
df =df.dropna()

In [8]:
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [9]:
df['QScore'].replace(['1A', '2A'])
df.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [10]:
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)

In [11]:
import sklearn.utils

In [12]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    224
Name: QScore, dtype: int64

In [13]:
 data_df = data_df.drop(columns=['country_code','country','year'])
 X = data_df.drop(columns='QScore')
 y = data_df['QScore']

In [14]:
#split the data into training and test sets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
y_train.value_counts()

3A    249
2A    152
Name: QScore, dtype: int64

In [15]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 1)
x_train_balanced, y_balanced = smote.fit_sample(x_train,y_train)
x_train_balanced = pd.DataFrame(x_train_balanced,columns=x_train.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

In [18]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df=pd.DataFrame(normalised_test_df,columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [19]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

ML Classification Lesson 2


In [20]:
#confusion matrix
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score,confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A','3A'])
cnf_mat 

array([[58, 14],
       [73, 28]])

In [21]:
#cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5,scoring='f1_macro')
scores

array([0.38945051, 0.48752228, 0.54526664, 0.44497508, 0.50902184])

In [22]:
#accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy: {}'.format(round(accuracy*100),2))

Accuracy: 50


In [23]:
#F1-Score
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))

F1: 57


In [24]:
#K-fold Cross Validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
  y_train, y_test = y_balanced[train_index],y_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)

  #save result to list
  f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test),pos_label='2A')*100)
f1_scores

[49.58677685950414,
 53.96825396825397,
 54.96183206106869,
 59.541984732824424,
 0.0]

In [25]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
  y_train, y_test  = y_balanced[train_index], y_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
  #save result to list
  f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)
  
f1_scores

[49.5049504950495, 67.1875, 55.17241379310344, 54.86725663716815, 62.5]

In [26]:
#LeaveOneOut
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score

49.79919678714859

In [27]:
# Tree-Based Methods and The Support Vector Machine
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

QUIZ: CLASSIFICATION


In [28]:
#import the libraries
import pandas as pd
import numpy as np


In [29]:
import urllib.request
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv'
filename = 'Data_for_UCI_named.csv'
urllib.request.urlretrieve(url, filename)

('Data_for_UCI_named.csv', <http.client.HTTPMessage at 0x7f47cf1a20d0>)

In [30]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.to_csv('dataset.csv')

In [31]:
#load the dataset
df = pd.read_csv('dataset.csv', low_memory=False)
df.head()

,Unnamed: 0,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [32]:
#drop the unnamed and stab columns

# df.drop('Unnamed: 0', axis = 1, inplace = True)
df.drop('stab', axis = 1, inplace = True)

In [33]:
df.head()

,Unnamed: 0,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [34]:
#check distribution of target variable
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [35]:
#checking for null values in the dataset
df.isna().sum()

Unnamed: 0    0
tau1          0
tau2          0
tau3          0
tau4          0
p1            0
p2            0
p3            0
p4            0
g1            0
g2            0
g3            0
g4            0
stabf         0
dtype: int64

In [36]:
#reset the dataframe index
df = df.reset_index(drop = True)

In [37]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [38]:
#reshuffle the dataset for randomness
import sklearn.utils
df = sklearn.utils.shuffle(df)
df = df.reset_index(drop=True)
df.shape
df.stabf.value_counts()


unstable    6380
stable      3620
Name: stabf, dtype: int64

In [39]:
#One of the feature is categorical, so we need to encode it ahead 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df.stabf = encoder.fit_transform(df.stabf)

In [40]:
df.stabf

0       1
1       0
2       1
3       1
4       0
       ..
9995    1
9996    1
9997    1
9998    1
9999    0
Name: stabf, Length: 10000, dtype: int64

In [41]:
X = df.drop(columns='stabf')
y = df['stabf']

In [42]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
y_train.value_counts()

1    5082
0    2918
Name: stabf, dtype: int64

In [45]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)
x_train_balanced = pd.DataFrame(x_train_balanced,columns=x_train.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [46]:
#min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced)
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.columns)
normalised_train_df = x_train_balanced

In [47]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.columns)
normalised_test_df = x_test

In [48]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
#returns

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
# Tree-Based Methods and The Support Vector Machine
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [59]:
n_estimators =[50,100,300,500,1000]
min_samples_split = [2,3,5,7,9]
min_samples_leaf = [1,2,4,6,8]
max_features = ['auto','sqrt','log2',None]

grid_param = {
    'n_estimators':n_estimators,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,
}

In [62]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score


#Extra tree classifier
ext_tree = ExtraTreesClassifier(random_state=1)
ext_tree = RandomizedSearchCV(estimator=ext_tree,
                         param_distributions = grid_param,random_state=1)

ext_tree.fit(normalised_train_df,y_balanced)
print(ext_tree.best_params_)

{'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'auto'}


In [71]:
#Extra tree classifier
ext_tree = ExtraTreesClassifier(random_state=1)
ext_tree = RandomizedSearchCV(estimator=ext_tree,
                         param_distributions = grid_param,cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1,random_state=1)

ext_tree.fit(normalised_train_df,y_balanced)
print(ext_tree.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.0min finished


{'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'auto'}


In [70]:
#ExtraTrees Classifier with no hyperparameter

ext_t = ExtraTreesClassifier(random_state=1)
ext_t.fit(normalised_train_df, y_balanced)


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [82]:

#RandomForestClassifier

RFC = RandomForestClassifier(random_state=1)
RFC.fit(normalised_train_df, y_balanced)

y_pred = RFC.predict(normalised_test_df)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.60%


In [80]:
import xgboost as xgb
xg_cla = xgb.XGBClassifier(random_state=1)
xg_cla.fit(normalised_train_df, y_balanced)


# model = XGBClassifier()
# model.fit(X_train, y_train)
# make predictions for test data
y_pred = xg_cla.predict(normalised_test_df)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.35%


In [81]:
import lightgbm as ltb
ltb_cla = ltb.LGBMClassifier(random_state=1)
ltb_cla.fit(normalised_train_df, y_balanced)

y_pred = ltb_cla.predict(normalised_test_df)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.20%


In [77]:
#Accuracy
# accuracy = accuracy_score(y_true = y_test, y_pred = new_predictions)
